# 01 - Dati e profili social

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import re

Ho scaricato le 12 tabelle dei giocatori più preziosi da Transfermarkt.

https://www.transfermarkt.it/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&plus=1

Siccome non è possibile fare un ciclo sull'indirizzo (vengono mostrati 25 risultati per pagina, quindi in totale sono 12 tabelle) ho scaricato e salvato in un file HTML le 12 tabelle così da poter rendere replicabile la prima parte del notebook (infatti i seguaci Instagram, Facebook, etc sono in **continuo** - praticamente istantaneo - cambiamento).

In [2]:
# leggo il file html e lo converto in stringa con la funzione .read()
f = open('tables/top300_values_14122018.html', 'r', encoding="utf8").read()

# tratto la stringa html esattamente come un sito qualunque con BeautifulSoup
html_tables = BS(f, 'lxml')

In [3]:
# estraggo dalle tabelle il maggior numero di informazioni possibile

id_player = [x['id'] for x in html_tables.find_all('a', attrs={'class': 'spielprofil_tooltip'})]
nomi = [x.text for x in html_tables.find_all('a', attrs={'class': 'spielprofil_tooltip'})]
eta = [x.find_all('td')[5].text for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
nazioni = [x.find_all('td')[6].find('img')['title'] for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
squadre = [x.find_all('td')[7].find('img')['alt'] for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
ruoli2 = [x.find_all('tr')[1].text for x in html_tables.find_all('table', attrs={'class': 'inline-table'})]
valori = [int(x.find('b').text.split(',')[0])*1000000 for x in html_tables.find_all('td', attrs={'class': 'rechts'})]

Voglio tenere sia il ruolo in generale, sia il ruolo nel dettaglio:

In [4]:
attaccante = ['Ala destra', 'Ala sinistra', 'Punta', 'Seconda punta']
centrocampista = ['Centrale', 'Centrocampista', 'Mediano', 'Trequartista']
difensore = ['Difensore', 'Terzino']

ruoli = ['Portiere']*len(ruoli2)

k = 0
for x in ruoli2:
    for r in attaccante:
        if r in x:
            ruoli[k] = 'Attaccante'
    for r in centrocampista:
        if r in x:
            ruoli[k] = 'Centrocampista'
    for r in difensore:
        if r in x:
            ruoli[k] = 'Difensore'
    k+=1
set(ruoli)

{'Attaccante', 'Centrocampista', 'Difensore', 'Portiere'}

In [5]:
df_dict = {
    'id':id_player,
    'nome':nomi,
    'eta':eta,
    'nazione':nazioni,
    'ruolo':ruoli,
    'ruolo_detail':ruoli2,
    'squadra':squadre,
    'valore':valori,
}

df = pd.DataFrame(df_dict)
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000


## Social Data (followers, likes, etc.)

Carico il dataframe con tutte le URLs dei tre social in questione

In [6]:
table_url = pd.read_csv('tables/Top300_Urls - Sheet1.csv')
table_url.head()

,name,transfermarkt,instagram,facebook,twitter
0,Kylian Mbappé,https://www.transfermarkt.it/kylian-mbappe/pro...,https://www.instagram.com/k.mbappe/,https://www.facebook.com/kylianmbappeofficiel/,https://twitter.com/KMbappe
1,Neymar,https://www.transfermarkt.it/neymar/profil/spi...,https://www.instagram.com/neymarjr,https://www.facebook.com/neymarjr,https://twitter.com/neymarjr
2,Lionel Messi,https://www.transfermarkt.it/lionel-messi/prof...,https://www.instagram.com/leomessi,https://www.facebook.com/LeoMessi,NaN
3,Mohamed Salah,https://www.transfermarkt.it/mohamed-salah/pro...,https://www.instagram.com/mosalah/,https://www.facebook.com/momosalah,https://twitter.com/MoSalah
4,Harry Kane,https://www.transfermarkt.it/harry-kane/profil...,https://www.instagram.com/harrykane/,https://www.facebook.com/harrykaneofficial,https://twitter.com/HKane


### Funzione che esegue scraping sui profili social

In [7]:
# getFollowers() prende in input una serie di link e il nome del social network
# restituisce come output una lista di interi pari al numero di followers/likes

def getFollowers(pdSeries,socialName,Output=False):
    if Output:
        print('1'+'-'*98+'100%'); 
    k = 1
    
    hs = {"User-Agent":"Mozilla/5.0"}
    followers = []
    errors = []
    
    # ciclo for all'interno della serie, u = url
    
    for i,u in pdSeries.iteritems():
        
        # se u=url non è nan (float):
         
        if not isinstance(u, float):
            
            # siccome temo che a volte requests non funzioni,
            # gli dico di provare n volte a scaricare la pagina
            # controllo che l'output sia un intero;
            # finché resta stringa vuol dire che qualcosa è andato storto
            
            s = ''   # per il controllo che sia un intero
            n = 1    # per fare massimo n prove
            
            
            while(isinstance(s, str) & (n < 4)):                
                try:
                    page = requests.get(u).text
                    
                    # scraping da INSTAGRAM
                    if socialName == 'Instagram':
                        s = int(re.findall(r'"edge_followed_by":{"count":+(\d*)', page)[0])
                        
                    # scraping da FACEBOOK
                    elif socialName == 'Facebook':
                        #s = int(re.findall(r'<div>Piace\sa\s+([\d|.]*)', page)[0].replace('.', ''))
                        s = int(re.findall(r'(\d*)\s+personnes+\s+aiment+\s+ça',re.sub('\xa0','',page))[0])
                    # scraping da TWITTER
                    elif socialName == 'Twitter':
                        a = BS(page,'lxml').find('a',attrs={'class':'ProfileNav-stat','data-nav':'followers'})['title']
                        s = int(a.split(' ')[0].replace('.',''))
                except: 
                    s = ''
                    n += 1
            
            # se anche dopo 3 volte non è riuscito ad estrarre il numero di followers
            # salvo dentro errori l'url problematica
            # e appendo a followers un errore
            
            
            if (isinstance(s, str)):
                errors.append(u)
                followers.append('errore')
                
            if len(errors) > 30:
                break
            
            # altrimenti è tutto ok e appendo il numero di followers
            
            else:
                followers.append(s)
        
        # l'url può essere np.nan (cioè float). in tal caso appendi np.nan
        
        else:
            followers.append(np.nan)
              
        if (k%3==0):
            if Output:
                print('=', end='')
        k += 1
    
    # prima printo gli errori e poi
    # ritorno la lista di followers
    print('\n');  
    
    if Output:
        print(errors)
    
    return(followers)

### *Instagram*

Spiegazione di cosa fa la formula quando `socialName='Instagram'`.

Per estrarre il numero esatto di seguaci, utilizzo una **regular expression** (pacchetto `re`):

`re.findall(r'"edge_followed_by":{"count":+(\d*)', page)[0]`

In sostanza noto che nel codice HTML di ogni profilo (contenuto in `page`) il numero di seguaci è preceduto dalla stringa: `"edge_followed_by":{"count":`

Quindi chiedo di estrarre - indicando tra parentesi tonde `( )` - tutti i numeri in sequenza - indicato dal `\d*` - specificando appunto la stringa che precede.

`findall` restituisce una lista, quindi prendo l'elemento `[0]`.

Questa è una via alternativa - **molto** rapida ed efficace - all'uso di BeautifulSoup.

In [8]:
df['instagram_followers'] =  getFollowers(table_url.instagram, 'Instagram', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [10]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0


In [11]:
df[df.instagram_followers.isnull()]

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers
31,65230,Sergio Busquets,30,Spagna,Centrocampista,Mediano,FC Barcellona,80000000,NaN
116,357565,Rodri,22,Spagna,Centrocampista,Mediano,Atlético de Madrid,40000000,NaN
220,50174,Danny Rose,28,Inghilterra,Difensore,Terzino sinistro,Tottenham Hotspur,30000000,NaN
260,229005,Abdou Diallo,22,Francia,Difensore,Difensore centrale,Borussia Dortmund,25000000,NaN


In totale ci sono 4 NaN: il numero è corretto.

### *Facebook*

Utilizzo la stessa tecnica che è stata usata per Instagram: una regular expression.

`re.findall(r'<div>Piace\sa\s+([\d|.]*)', page)[0]`

In questo caso il numero di likes è sempre preceduto da un `<div>` e da "Piace a ". 

Sfrutto questa particolarità per estrarre i valori, che vengono poi trasformati in intero dopo aver rimosso i punti dalle stringhe.

In [13]:
df['facebook_followers'] =  getFollowers(table_url.facebook, 'Facebook', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [14]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers,facebook_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0,1639467.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0,60699722.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0,89898268.0
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0,11016135.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0,1660408.0


### *Twitter*

In questo caso, per estrarre i followers, ho utilizzato BeautifulSoup, in quanto risulta il modo più agevole (seppur più lento).

In [17]:
df['twitter_followers'] =  getFollowers(table_url.twitter, 'Twitter', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [18]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers,facebook_followers,twitter_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0,1639467.0,2252200.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0,60699722.0,41636656.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0,89898268.0,NaN
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0,11016135.0,7457524.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0,1660408.0,2453183.0


In [19]:
df.to_csv('columns/df1.csv', index=False, encoding='ansi')